# Puzzle generation

In [25]:
import numpy as np
import json
from typing import List, Tuple

NUMBER_OF_EASY = 100
NUMBER_OF_MEDIUM = 100
NUMBER_OF_HARD = 100

def extract_constraints(grid: np.ndarray) -> Tuple[List[List[int]], List[List[int]]]:
    def get_line_constraints(line):
        constraints = []
        count = 0
        for val in line:
            if val == 1:
                count += 1
            elif count > 0:
                constraints.append(count)
                count = 0
        if count > 0:
            constraints.append(count)
        return constraints or [0]

    row_constraints = [get_line_constraints(row) for row in grid]
    col_constraints = [get_line_constraints(col) for col in grid.T]
    return row_constraints, col_constraints

def generate_puzzles(size: int, count: int) -> List[dict]:
    puzzles = []
    while len(puzzles) < count:
        solution = np.random.randint(0, 2, (size, size))
        row_cons, col_cons = extract_constraints(solution)
        puzzles.append({
            "row": row_cons,
            "col": col_cons,
            "solution": solution.tolist()
        })
    return puzzles

def save_to_file(puzzles: List[dict], filename: str):
    with open(filename, 'w') as f:
        for puzzle in puzzles:
            f.write(json.dumps(puzzle) + '\n')


save_to_file(generate_puzzles(5, NUMBER_OF_EASY), "small.txt")
save_to_file(generate_puzzles(10, NUMBER_OF_MEDIUM), "medium.txt")
save_to_file(generate_puzzles(15, NUMBER_OF_HARD), "large.txt")


# Code for getting solution statistics

In [26]:
import numpy as np
import json

def check(grid, row_constraints, col_constraints):
    return (
        all(extract_blocks(row) == r for row, r in zip(grid, row_constraints)) and
        all(extract_blocks(col) == c for col, c in zip(grid.T, col_constraints))
    )

def extract_blocks(arr):
    """Helper to extract lengths of consecutive 1s in a binary array."""
    return [sum(g) for g in np.split(arr, np.where(arr == 0)[0]) if sum(g) > 0]

def puzzle_accuracy(grid, row_constraints, col_constraints):
    """Return the average proportion of correct rows and columns."""
    row_score = sum(extract_blocks(row) == r for row, r in zip(grid, row_constraints))
    col_score = sum(extract_blocks(col) == c for col, c in zip(grid.T, col_constraints))
    return (row_score + col_score) / (len(row_constraints) * 2)

def constraints_accuracy(grid, row_constraints, col_constraints):
    """Return the proportion of satisfied constraints (correct rows + correct cols)."""
    correct_rows = sum(extract_blocks(row) == r for row, r in zip(grid, row_constraints))
    correct_cols = sum(extract_blocks(col) == c for col, c in zip(grid.T, col_constraints))
    return (correct_rows + correct_cols) / (len(row_constraints) + len(col_constraints))

def save_result(results: dict, filename: str) -> None:
    with open(filename, "w") as f:
        json.dump(results, f, indent=2)


# Genetic Algorithm

In [27]:
import random
import numpy as np
from numpy.typing import NDArray
import time
import json

# Set global vars for the solver
mutation_rate = 0.01
generations = 100
population_size = 100

# Load puzzles
with open("small.txt") as f:
    small = [json.loads(line) for line in f]
with open("medium.txt") as f:
    medium = [json.loads(line) for line in f]
with open("large.txt") as f:
    large = [json.loads(line) for line in f]

results = {"small": [], "medium": [], "large": []}

def create_individual() -> list[int]:
    """Generate a random individual (binary grid representation)."""
    return [random.randint(0, 1) for _ in range(grid_size**2)]

def evaluate(individual: list[int]) -> int:
    """Evaluate the fitness of an individual based on row and column constraints."""
    grid = np.array(individual).reshape((grid_size, grid_size))
    score = 0  # Higher score means a better match to constraints

    # Evaluate row constraints
    for i, row in enumerate(grid):
        blocks = [sum(g) for g in np.split(row, np.where(row == 0)[0]) if sum(g) > 0]
        if blocks == row_constraints[i]:  
            score += 1  

    # Evaluate column constraints
    for j, col in enumerate(grid.T):  # Transpose to iterate over columns
        blocks = [sum(g) for g in np.split(col, np.where(col == 0)[0]) if sum(g) > 0]
        if blocks == col_constraints[j]:  
            score += 1  

    return score  # Higher score means better constraint satisfaction

def select(population: list[list[int]]) -> list[list[int]]:
    """Select the top-performing individuals for reproduction."""
    population.sort(key=evaluate, reverse=True)  # Sort by fitness (descending order)
    return population[:population_size // 2]  # Retain the top 50% of individuals

def crossover(parent1: list[int], parent2: list[int]) -> tuple[list[int], list[int]]:
    """Perform single-point crossover between two parents."""
    point = random.randint(0, grid_size**2 - 1)  # Choose a crossover point
    child1 = parent1[:point] + parent2[point:]  
    child2 = parent2[:point] + parent1[point:]  
    return child1, child2

def mutate(individual: list[int]) -> None:
    """Apply mutation to an individual with a certain probability."""
    for i in range(len(individual)):
        if random.random() < mutation_rate:  
            individual[i] = 1 - individual[i]  # Flip the bit (0 → 1, 1 → 0)

def genetic_algorithm() -> NDArray[np.long]:
    """Solve the Picross puzzle using a genetic algorithm."""
    population = [create_individual() for _ in range(population_size)]

    for generation in range(generations):
        population = select(population)  # Select the fittest individuals

        # If an individual satisfies all constraints, terminate early
        if evaluate(population[0]) == grid_size * 2:
            break

        next_generation = population.copy()

        # Generate offspring until the population size is restored
        while len(next_generation) < population_size:
            parent1, parent2 = random.sample(population, 2)  # Select two parents
            child1, child2 = crossover(parent1, parent2)  # Perform crossover
            mutate(child1)  # Apply mutation
            mutate(child2)  
            next_generation.extend([child1, child2])  # Add offspring to the population
        
        population = next_generation[:population_size]  # Ensure population size remains constant
        
        best_individual = max(population, key=evaluate)  # Track the best solution found

    # Display the best solution found
    best_solution = np.array(best_individual).reshape((grid_size, grid_size))
    
    return best_solution

# Run the genetic algorithm to solve the Picross puzzle
for size_name, puzzles in [("small", small), ("medium", medium), ("large", large)]:
    for puzzle in puzzles:
        row_constraints = puzzle["row"]
        col_constraints = puzzle["col"]
        grid_size = len(row_constraints)

        start = time.time()
        grid = genetic_algorithm()
        end = time.time()
        print(end - start)
        
        correct = check(grid, row_constraints, col_constraints)
        p_acc = puzzle_accuracy(grid, row_constraints, col_constraints)
        c_acc = constraints_accuracy(grid, row_constraints, col_constraints)

        results[size_name].append({
            "solved": correct,
            "time": end - start,
            "puzzle_accuracy": p_acc,
            "constraints_accuracy": c_acc
        })

# Save results
save_result(results, "GA-results.json")


0.4608457088470459
1.7116780281066895
1.7510490417480469
0.3838765621185303
1.8648381233215332
0.2927677631378174
1.7302405834197998
0.5688352584838867
1.8478586673736572
1.8831570148468018
1.5762560367584229
1.8236777782440186
1.6097650527954102
1.764064073562622
1.831481695175171
0.30961036682128906
1.9220876693725586
1.9430732727050781
0.21037530899047852
1.8239006996154785
1.8367056846618652
1.8679554462432861
1.8886620998382568
0.3373844623565674
1.822866678237915
0.21002435684204102
1.8561301231384277
1.8135349750518799
1.8191282749176025
0.3535020351409912
1.9357986450195312
1.8021295070648193
1.8357107639312744
0.32087016105651855
1.7155046463012695
1.7505435943603516
1.858659267425537
1.8586838245391846
0.2924206256866455
0.32405567169189453
0.23317837715148926
0.24632835388183594
1.881702184677124
0.46762585639953613
0.29915881156921387
0.45359015464782715
1.7272961139678955
1.8263657093048096
1.847121238708496
0.26799511909484863
0.2766087055206299
1.784393072128296
1.862761

# Simulated Annealing

In [28]:
import random
import numpy as np
from numpy.typing import NDArray
import time
import json

# Set global vars for the solver
initial_temperature = 10.0
cooling_rate = 0.999
iterations = 10000

# Load puzzles
with open("small.txt") as f:
    small = [json.loads(line) for line in f]
with open("medium.txt") as f:
    medium = [json.loads(line) for line in f]
with open("large.txt") as f:
    large = [json.loads(line) for line in f]

results = {"small": [], "medium": [], "large": []}

def evaluate(grid: NDArray[np.long]) -> int:
    """
    Calculate the fitness score of a grid based on row and column constraints.
    A higher score indicates a better match to the constraints.
    """
    score = 0  # Initialize score
    
    # Evaluate row constraints
    for i, row in enumerate(grid):
        # Find contiguous blocks of 1s in the row
        blocks = [sum(g) for g in np.split(row, np.where(row == 0)[0]) if sum(g) > 0]
        if blocks == row_constraints[i]:  
            score += 1  

    # Evaluate column constraints
    for j, col in enumerate(grid.T):  # Transpose to iterate over columns
        blocks = [sum(g) for g in np.split(col, np.where(col == 0)[0]) if sum(g) > 0]
        if blocks == col_constraints[j]:  
            score += 1  

    return score  # Higher score means better constraint satisfaction

def mutate(grid: NDArray[np.long]) -> NDArray[np.long]:
    """
    Create a new grid by flipping a random cell (mutating the grid).
    """
    new_grid = grid.copy()  # Copy the current grid
    i, j = random.randint(0, grid_size - 1), random.randint(0, grid_size - 1)  # Select a random cell
    new_grid[i, j] = 1 - new_grid[i, j]  # Flip the bit (0 → 1, 1 → 0)
    return new_grid

def simulated_annealing() -> NDArray[np.long]:
    """
    Solve the Picross puzzle using the simulated annealing algorithm.
    """
    # Initialize a random grid with binary values (0 or 1)
    current_grid = np.random.randint(0, 2, (grid_size, grid_size))
    current_score = evaluate(current_grid)  # Compute its initial fitness score
    temperature = initial_temperature  # Set the initial temperature
    
    # Keep track of the best solution found
    best_grid, best_score = current_grid, current_score

    for step in range(iterations):
        # Generate a new candidate solution by mutating the current grid
        new_grid = mutate(current_grid)
        new_score = evaluate(new_grid)

        # If the new grid is better, accept it unconditionally
        if new_score > current_score:
            current_grid, current_score = new_grid, new_score
        else:
            # Otherwise, accept with a probability determined by the temperature
            delta = new_score - current_score
            if random.random() < np.exp(delta / temperature):  
                current_grid, current_score = new_grid, new_score

        # Update the best solution if the new one is the best so far
        if new_score > best_score:
            best_grid, best_score = new_grid, new_score

        # Reduce the temperature over time
        temperature *= cooling_rate

        # Stop early if a perfect solution is found
        if best_score == grid_size * 2:
            break

    return best_grid

# Run the simulated annealing algorithm to solve the Picross puzzle
for size_name, puzzles in [("small", small), ("medium", medium), ("large", large)]:
    for puzzle in puzzles:
        row_constraints = puzzle["row"]
        col_constraints = puzzle["col"]
        grid_size = len(row_constraints)

        start = time.time()
        grid = simulated_annealing()
        end = time.time()
        print(end - start)
        correct = check(grid, row_constraints, col_constraints)
        p_acc = puzzle_accuracy(grid, row_constraints, col_constraints)
        c_acc = constraints_accuracy(grid, row_constraints, col_constraints)

        results[size_name].append({
            "solved": correct,
            "time": end - start,
            "puzzle_accuracy": p_acc,
            "constraints_accuracy": c_acc
        })

# Save results
save_result(results, "SA-results.json")


0.33966732025146484
0.33174681663513184
0.9520461559295654
0.27912044525146484
0.9729557037353516
0.3052363395690918
0.9114809036254883
0.9151515960693359
0.9388139247894287
0.9876806735992432
0.8820996284484863
0.3583989143371582
0.8905160427093506
0.9356169700622559
0.955904483795166
0.8610222339630127
0.9269046783447266
0.9337503910064697
0.38367700576782227
0.303194522857666
0.9167253971099854
0.9115245342254639
0.9233002662658691
0.269911527633667
0.9120075702667236
0.31885719299316406
0.2520172595977783
0.2966172695159912
0.906527042388916
0.31092238426208496
0.3470942974090576
0.352158784866333
0.31635522842407227
0.3240971565246582
0.3685445785522461
0.9496333599090576
0.9575910568237305
0.9275383949279785
0.948986291885376
0.9532420635223389
0.3733224868774414
0.29502320289611816
0.9597015380859375
0.9000961780548096
0.9519696235656738
0.32671117782592773
0.33429431915283203
0.3448140621185303
0.29279041290283203
0.9285626411437988
0.9433135986328125
0.2668757438659668
0.96613

# Results Analysis

In [29]:
import json

def analyze(file: str):
    with open(file) as f:
        data = json.load(f)

    for size, records in data.items():
        total = len(records)
        solved = sum(1 for r in records if r["solved"])
        avg_time = sum(r["time"] for r in records) / max(total, 1)
        avg_puzzle_acc = sum(r.get("puzzle_accuracy", 0) for r in records) / max(total, 1)
        avg_constraint_acc = sum(r.get("constraints_accuracy", 0) for r in records) / max(total, 1)

        print(f"{file} - {size}:")
        print(f"  Accuracy: {solved / max(total, 1) * 100:.1f}%")
        print(f"  Avg puzzle accuracy: {avg_puzzle_acc * 100:.2f}%")
        print(f"  Avg constraint accuracy: {avg_constraint_acc * 100:.2f}%")
        print(f"  Avg time: {avg_time:.4f}s\n")

analyze("GA-results.json")
analyze("SA-results.json")

GA-results.json - small:
  Accuracy: 37.0%
  Avg puzzle accuracy: 87.80%
  Avg constraint accuracy: 87.80%
  Avg time: 1.2981s

GA-results.json - medium:
  Accuracy: 0.0%
  Avg puzzle accuracy: 51.40%
  Avg constraint accuracy: 51.40%
  Avg time: 5.5239s

GA-results.json - large:
  Accuracy: 0.0%
  Avg puzzle accuracy: 25.83%
  Avg constraint accuracy: 25.83%
  Avg time: 12.9455s

SA-results.json - small:
  Accuracy: 49.0%
  Avg puzzle accuracy: 92.40%
  Avg constraint accuracy: 92.40%
  Avg time: 0.6363s

SA-results.json - medium:
  Accuracy: 2.0%
  Avg puzzle accuracy: 73.85%
  Avg constraint accuracy: 73.85%
  Avg time: 3.3614s

SA-results.json - large:
  Accuracy: 0.0%
  Avg puzzle accuracy: 42.03%
  Avg constraint accuracy: 42.03%
  Avg time: 10.3831s

